In [ ]:
# install some of the modules first
pip install mouse
pip install keyboard
pip install pickle

In [2]:
import mouse
import keyboard
import pickle

file = input('Mouse movement name?? :\n')
file = file + '.txt'

# simple recording of mouse events

# put to list
mouse_events = []

# start recording
mouse.hook(mouse_events.append)


#kb = keyboard.record(until ='Esc')

#press Esc to kill the event
if not keyboard.wait("Esc"):
    mouse.unhook_all()
    # save the recording    
    with open(file, 'wb') as fh:
       pickle.dump(mouse_events, fh)


Mouse movement name?? :
mouse1


In [ ]:
# open the recording
pickle_off = open (file, "rb")
emp = pickle.load(pickle_off)


# replay the events
mouse.play(emp)

# Enhanced version to combine mouse and keyboard

In [15]:

import mouse
import keyboard            
import pickle
import pandas as pd

mouse_events = []


mouse.hook(mouse_events.append)
keyboard.start_recording()

keyboard.wait("Esc")

mouse.unhook(mouse_events.append)
keyboard_events = keyboard.stop_recording()


# put mouse results (click only) to list
list = []

for i in range(len(mouse_events)):
    if mouse_events[i][0] == 'down':
        # pick up previus coordinate
        #row = i - 1
        #x = mouse_events[row][0]
        #y = mouse_events[row][1]     
        #list.append([x,y])
        
        for chk in range(1,5):
        #    # check previous 5 rows until you dont see 'down' or 'up' key
            row = i - chk
            if mouse_events[row][0] not in ('down','up'):
                 x = mouse_events[row][0]
                 y = mouse_events[row][1]     
                 z = mouse_events[row][2]
                 list.append(['Mouse Click Event',x,y,z])
                 break   
                
# put keyboard results to list
string = []
stroke = []
finalword = []

for m in range(len(keyboard_events)):
    mystring = str(keyboard_events[m])
    letter = mystring[14:15]
    if 'down' in mystring:
        if 'backspace' in mystring:
            #print (string)
            string.pop()
            #print (string)
            #list.append(['Keyboard Event','backspace'])
        elif 'space' in mystring:
             #list.append(['Keyboard Event','space'])
             string.append(' ')
        elif 'enter' in mystring:
               
            if len(string) != 0 :
                finalword = ''.join(string) 
                list.append(['Keyboard Write',finalword])
                list.append(['Keyboard Event','enter'])
                string = []
            
            #finalword = []
            #string.append(' Enter ')
        elif 'tab' in mystring:
            #list.append(['Keyboard Event','enter'])
            #string.append(' Tab ')
            if len(string) != 0 :
                finalword = ''.join(string) 
                list.append(['Keyboard Write',finalword])
                list.append(['Keyboard Event','tab'])
                string = []
            
        elif 'shift' in mystring:
            # shift means nothing for this logic as it is just uppercase or lowercase
            letter = ''
            
        elif 'esc' in mystring:
            #list.append(['Ignore','esc'])
            #print (mystring)
            if len(string) != 0 :
                finalword = ''.join(string) 
                list.append(['Keyboard Write',finalword])
                list.append(['Keyboard Event','enter'])
                string = []
        else:
            string.append(letter)
            
            
# add duration to the list

for index, i in enumerate(list):
   try :
    if index != 0:
        now = list[index][3]
        before = list[index-1][3]
        duration = now - before
        list[index].append(duration)
    else:
        list[index].append(0)
   except:
        break
        
        
# tidy up the sequence between mouse and keyboards into a list

import pyautogui as s
import time

tmp = []
newlist = []
#count = 0

for index, i in enumerate(list):
    if 'Mouse' in i[0]:             # mouse in string 
        time = i[4]                 # time column
        if time < 2 and tmp == []:
            newlist.append(i)
        else:
            # if time > 2 seconds, move them into tmp list
            tmp.append(i)
    elif 'Keyboard Write' in i[0]:
        newlist.append(i)
    elif 'Keyboard Event' in i[0]:
        newlist.append(i)
        
        
        # first check if it is a tab event
        if i[0][1] == 'tab':
            # add the next two events to the list
            newlist.append(i+1)
            newlist.append(i+2)
        
       
        
        else:
        # after keyboard events will bring back those mouse events
        # from tmp list after the keyboards.
        # logic will loop until the next mouse event > 2 seconds
        # at the moment assumption is typing will be more than 2 seconds.
            for i in range(len(tmp)):

                if i == 0:     # first event, straight back to the main list
                    count = 0
                    newlist.append(tmp[count])
                    count += 1
                    #tmp.pop(i)
                    #i = i - 1
                elif tmp[i][4] < 2:      # if less than 2 seconds, keep adding to main list
                    newlist.append(tmp[count])
                    count += 1
                    #tmp.pop(i)
                    #i = i -1
                elif tmp[i][4] > 2:      # when the next keyboard even found (> 2 seconds), exit the loop
                    break
        
# if no keyboard events, length of new list will be shorter than the original list
# hence just use the original list

if len(newlist) != len(list):
    newlist = list
            
# save the recording    
with open('record.txt', 'wb') as fh:
   pickle.dump(newlist, fh)


# open the recording
pickle_off = open ('record.txt', "rb")
newlist = pickle.load(pickle_off)



# put to dataframes
x1 = pd.DataFrame(list)
x2 = pd.DataFrame(newlist)
x1.columns = ['Event','X','Y','Time','Duration']
x2.columns = ['Event','X','Y','Time','Duration']
print ('Initial List \n',x1,'\n\n','New list \n',x2)

   

Initial List 
                Event    X    Y          Time  Duration
0  Mouse Click Event  978  761  1.637320e+09  0.000000
1  Mouse Click Event  306   22  1.637320e+09  2.051286
2  Mouse Click Event  601  246  1.637320e+09  1.568961
3  Mouse Click Event  783  240  1.637320e+09  2.251745
4  Mouse Click Event  743  287  1.637320e+09  2.951008 

 New list 
                Event    X    Y          Time  Duration
0  Mouse Click Event  978  761  1.637320e+09  0.000000
1  Mouse Click Event  306   22  1.637320e+09  2.051286
2  Mouse Click Event  601  246  1.637320e+09  1.568961
3  Mouse Click Event  783  240  1.637320e+09  2.251745
4  Mouse Click Event  743  287  1.637320e+09  2.951008


In [16]:
# replay the click        
import pyautogui as s
import time

#newlist = list

for i in range(len(newlist)):
    if 'Mouse' in newlist[i][0]:
        x = newlist[i][1]
        y = newlist[i][2]
        s.moveTo(x,y,0.7)
        s.click()
        time.sleep(2.5)
    elif 'Write' in newlist[i][0]:
        #break
        #s.press('enter')
        s.write(newlist[i][1],0.3)
    elif 'tab' in newlist[i][1]:
        s.press('tab')
        time.sleep(0.8)
    elif 'enter' in newlist[i][1]:
        s.press('enter')
        time.sleep(2.5)

##### work in progress version 2, to fix tab event e.g combank but still not working


In [11]:
# v2 recording



import mouse
import keyboard            
import pickle

mouse_events = []


mouse.hook(mouse_events.append)
keyboard.start_recording()

keyboard.wait("Esc")

mouse.unhook(mouse_events.append)
keyboard_events = keyboard.stop_recording()


# put mouse results (click only) to list
list = []

for i in range(len(mouse_events)):
    if mouse_events[i][0] == 'down':
        # pick up previus coordinate
        #row = i - 1
        #x = mouse_events[row][0]
        #y = mouse_events[row][1]     
        #list.append([x,y])
        
        for chk in range(1,5):
        #    # check previous 5 rows until you dont see 'down' or 'up' key
            row = i - chk
            if mouse_events[row][0] not in ('down','up'):
                 x = mouse_events[row][0]
                 y = mouse_events[row][1]     
                 z = mouse_events[row][2]
                 list.append(['Mouse Click Event',x,y,z])
                 break   
                
# put keyboard results to list
string = []
stroke = []
finalword = []

for m in range(len(keyboard_events)):
    mystring = str(keyboard_events[m])
    letter = mystring[14:15]
    if 'down' in mystring:
        if 'backspace' in mystring:
            #print (string)
            string.pop()
            #print (string)
            #list.append(['Keyboard Event','backspace'])
        elif 'space' in mystring:
             #list.append(['Keyboard Event','space'])
             string.append(' ')
        elif 'enter' in mystring:
               
            if len(string) != 0 :
                finalword = ''.join(string) 
                list.append(['Keyboard Write',finalword])
                list.append(['Keyboard Event','enter'])
                string = []
            
            #finalword = []
            #string.append(' Enter ')
        elif 'tab' in mystring:
            #list.append(['Keyboard Event','enter'])
            #string.append(' Tab ')
            if len(string) != 0 :
                finalword = ''.join(string) 
                list.append(['Keyboard Write',finalword])
                list.append(['Keyboard Event','tab'])
                string = []
            
        elif 'shift' in mystring:
            # shift means nothing for this logic as it is just uppercase or lowercase
            letter = ''
            
        elif 'esc' in mystring:
            #list.append(['Ignore','esc'])
            #print (mystring)
            if len(string) != 0 :
                finalword = ''.join(string) 
                list.append(['Keyboard Write',finalword])
                list.append(['Keyboard Event','enter'])
                string = []
        else:
            string.append(letter)
            
            
# add duration to the list

for index, i in enumerate(list):
   try :
    if index != 0:
        now = list[index][3]
        before = list[index-1][3]
        duration = now - before
        list[index].append(duration)
    else:
        list[index].append(0)
   except:
        break
        
        
# tidy up the sequence between mouse and keyboards into a list

import pyautogui as s
import time

tmp = []
newlist = []
count = 0

for index, i in enumerate(list):
    if 'Mouse' in i[0]:             # mouse in string 
        time = i[4]                 # time column
        if time < 2 and tmp == []:
            newlist.append(i)
        else:
            # if time > 2 seconds, move them into tmp list
            tmp.append(i)
    
    elif 'Keyboard Write' in i[0] and i[1] != 'tab':
        if status != 'skip':
            newlist.append(i)
            status = ''
        else:
            status = ''
    elif 'Keyboard Event' in i[0] and i[1] != 'tab':
        if status != 'skip':
            newlist.append(i)
        else:
            status = ''
    elif 'Keyboard Event' in i[0] and i[1] == 'tab':
         newlist.append(i)
         newlist.append(list[index+1])
         newlist.append(list[index+2])
         status = 'skip'
        
    else:
    # after keyboard events will bring back those mouse events
    # from tmp list after the keyboards.
    # logic will loop until the next mouse event > 2 seconds
    # at the moment assumption is typing will be more than 2 seconds.
        for i in range(len(tmp)):

            if i == 0:     # first event, straight back to the main list
                newlist.append(tmp[count])
                count += 1
                #tmp.pop(i)
                #i = i - 1
            elif tmp[i][4] < 2:      # if less than 2 seconds, keep adding to main list
                newlist.append(tmp[count])
                count += 1
                #tmp.pop(i)
                #i = i -1
            elif tmp[i][4] > 2:      # when the next keyboard even found (> 2 seconds), exit the loop
                break
        
# if no keyboard events, length of new list will be shorter than the original list
# hence just use the original list

if len(newlist) != len(list):
    newlist = list
            
# save the recording    
with open('record.txt', 'wb') as fh:
   pickle.dump(newlist, fh)


# open the recording
pickle_off = open ('record.txt', "rb")
newlist = pickle.load(pickle_off)

   


In [238]:
list

[['Mouse Click Event', 1267, 16, 1637041477.5128713, 0],
 ['Mouse Click Event', 1493, 16, 1637041481.260487, 3.7476158142089844],
 ['Keyboard Write', 'smh'],
 ['Keyboard Event', 'enter'],
 ['Keyboard Write', 'news.com.au'],
 ['Keyboard Event', 'enter']]

In [239]:
newlist

[['Mouse Click Event', 1267, 16, 1637041477.5128713, 0],
 ['Keyboard Write', 'smh'],
 ['Keyboard Event', 'enter'],
 ['Mouse Click Event', 1493, 16, 1637041481.260487, 3.7476158142089844],
 ['Keyboard Write', 'news.com.au'],
 ['Keyboard Event', 'enter']]

In [90]:
tmp

[['Mouse Click Event', 1857, 103, 1637034558.8875306, 5.147812843322754],
 ['Mouse Click Event', 1669, 120, 1637034560.043803, 1.1562724113464355],
 ['Mouse Click Event', 836, 222, 1637034562.0071008, 1.9632978439331055]]

In [225]:
tmp = []
newlist = []


test = []

for index, i in enumerate(list):
    if 'Mouse' in i[0]:             # mouse in string 
        time = i[4]                 # time column
        if time < 2 and tmp == []:
            newlist.append(i)
        else:
            # if time > 2 seconds, move them into tmp list
            tmp.append(i)
    elif 'Keyboard Write' in i[0] and i[1] != 'tab':
        if status != 'skip':
            newlist.append(i)
            status = ''
        else:
            status = ''
    elif 'Keyboard Event' in i[0] and i[1] != 'tab':
        if status != 'skip':
            newlist.append(i)
        else:
            status = ''
    elif 'Keyboard Event' in i[0] and i[1] == 'tab':
         newlist.append(i)
         newlist.append(list[index+1])
         newlist.append(list[index+2])
         status = 'skip'


In [226]:
list

[['Mouse Click Event', 1263, 20, 1637035935.692013, 0],
 ['Mouse Click Event', 1896, 109, 1637035940.68884, 4.996826887130737],
 ['Mouse Click Event', 1696, 127, 1637035941.6992002, 1.0103602409362793],
 ['Mouse Click Event', 868, 225, 1637035942.813764, 1.1145639419555664],
 ['Keyboard Write', 'comm'],
 ['Keyboard Event', 'enter'],
 ['Keyboard Write', '11695328'],
 ['Keyboard Event', 'tab'],
 ['Keyboard Write', 'B4055ph!'],
 ['Keyboard Event', 'enter']]

In [228]:
newlist

[['Mouse Click Event', 1263, 20, 1637035935.692013, 0],
 ['Keyboard Event', 'enter'],
 ['Keyboard Write', '11695328'],
 ['Keyboard Event', 'tab'],
 ['Keyboard Write', 'B4055ph!'],
 ['Keyboard Event', 'enter'],
 ['Keyboard Event', 'enter']]

In [229]:
list

[['Mouse Click Event', 1263, 20, 1637035935.692013, 0],
 ['Mouse Click Event', 1896, 109, 1637035940.68884, 4.996826887130737],
 ['Mouse Click Event', 1696, 127, 1637035941.6992002, 1.0103602409362793],
 ['Mouse Click Event', 868, 225, 1637035942.813764, 1.1145639419555664],
 ['Keyboard Write', 'comm'],
 ['Keyboard Event', 'enter'],
 ['Keyboard Write', '11695328'],
 ['Keyboard Event', 'tab'],
 ['Keyboard Write', 'B4055ph!'],
 ['Keyboard Event', 'enter']]

In [ ]:
 else:
            # after keyboard events will bring back those mouse events
            # from tmp list after the keyboards.
            # logic will loop until the next mouse event > 2 seconds
            # at the moment assumption is typing will be more than 2 seconds.
             print ('im else')
             for i in range(len(tmp)):

                    if i == 0:     # first event, straight back to the main list
                        count = 0
                        newlist.append(tmp[count])
                        #tmp.pop(i)
                        #i = i - 1
                    elif tmp[i][4] < 2:      # if less than 2 seconds, keep adding to main list
                        count += 1
                        newlist.append(tmp[count])
                        #tmp.pop(i)
                        #i = i -1
                    elif tmp[i][4] > 2:      # when the next keyboard even found (> 2 seconds), exit the loop
                        break     
        
       

In [215]:
list
   

[['Mouse Click Event', 1263, 20, 1637035935.692013, 0],
 ['Mouse Click Event', 1896, 109, 1637035940.68884, 4.996826887130737],
 ['Mouse Click Event', 1696, 127, 1637035941.6992002, 1.0103602409362793],
 ['Mouse Click Event', 868, 225, 1637035942.813764, 1.1145639419555664],
 ['Keyboard Write', 'comm'],
 ['Keyboard Event', 'enter'],
 ['Keyboard Write', '11695328'],
 ['Keyboard Event', 'tab'],
 ['Keyboard Write', 'B4055ph!'],
 ['Keyboard Event', 'enter']]

In [216]:
newlist

[['Mouse Click Event', 1263, 20, 1637035935.692013, 0],
 ['Keyboard Write', 'comm'],
 ['Keyboard Event', 'enter'],
 ['Mouse Click Event', 1896, 109, 1637035940.68884, 4.996826887130737],
 ['Mouse Click Event', 1696, 127, 1637035941.6992002, 1.0103602409362793],
 ['Mouse Click Event', 868, 225, 1637035942.813764, 1.1145639419555664],
 ['Keyboard Write', '11695328'],
 ['Keyboard Write', 'B4055ph!'],
 ['Keyboard Event', 'enter'],
 ['Mouse Click Event', 1896, 109, 1637035940.68884, 4.996826887130737],
 ['Mouse Click Event', 1696, 127, 1637035941.6992002, 1.0103602409362793],
 ['Mouse Click Event', 868, 225, 1637035942.813764, 1.1145639419555664]]

In [206]:
list

[['Mouse Click Event', 1263, 20, 1637035935.692013, 0],
 ['Mouse Click Event', 1896, 109, 1637035940.68884, 4.996826887130737],
 ['Mouse Click Event', 1696, 127, 1637035941.6992002, 1.0103602409362793],
 ['Mouse Click Event', 868, 225, 1637035942.813764, 1.1145639419555664],
 ['Keyboard Write', 'comm'],
 ['Keyboard Event', 'enter'],
 ['Keyboard Write', '11695328'],
 ['Keyboard Event', 'tab'],
 ['Keyboard Write', 'B4055ph!'],
 ['Keyboard Event', 'enter']]

In [207]:
newlist

[['Mouse Click Event', 1263, 20, 1637035935.692013, 0],
 ['Keyboard Write', 'comm'],
 ['Keyboard Event', 'enter'],
 ['Mouse Click Event', 1896, 109, 1637035940.68884, 4.996826887130737],
 ['Mouse Click Event', 1696, 127, 1637035941.6992002, 1.0103602409362793],
 ['Mouse Click Event', 868, 225, 1637035942.813764, 1.1145639419555664],
 ['Keyboard Write', '11695328'],
 ['Keyboard Write', 'B4055ph!'],
 ['Keyboard Event', 'enter'],
 ['Mouse Click Event', 1896, 109, 1637035940.68884, 4.996826887130737],
 ['Mouse Click Event', 1696, 127, 1637035941.6992002, 1.0103602409362793],
 ['Mouse Click Event', 868, 225, 1637035942.813764, 1.1145639419555664]]

# Once recorded, you can add parameters to the list

1. Define the parameter in param
2. specific the location of the index tochange variable
3. store the new variable under x
4. put that x value to the list

Re-run the replay

In [36]:
# enter parameter and index to change
param = '2020-06-30'
tochange = newlist[3][1]

x = str(tochange) + ' ' + str(param)
newlist[3][1] = x

print (x)

test 2020-06-30
